In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
aqi_data_path = 'aqi_data.csv'
aqi_data = pd.read_csv(aqi_data_path)
aqi_data

,Start,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25,AQI_SO2,Overall_AQI,Overall_AQI_Smoothed
0,2004-03-01 00:00:00,19.318182,60.586957,9.959635,56.939394,69.716738,8.571429,69.716738,NaN
1,2004-03-01 01:00:00,18.181818,52.065217,9.959635,56.939394,69.716738,0.000000,69.716738,NaN
2,2004-03-01 02:00:00,17.803030,43.396226,9.959635,56.939394,69.716738,7.142857,69.716738,NaN
3,2004-03-01 03:00:00,17.329546,33.962264,9.959635,56.939394,69.716738,0.000000,69.716738,NaN
4,2004-03-01 04:00:00,17.045455,36.792453,9.959635,56.939394,69.716738,0.000000,69.716738,NaN
...,...,...,...,...,...,...,...,...,...
10028,2005-04-29 20:00:00,7.528409,63.782609,42.824074,22.029321,67.175608,8.571429,67.175608,81.827631
10029,2005-04-29 21:00:00,7.244318,64.847826,42.997685,22.029321,67.175608,7.142857,67.175608,80.992557
10030,2005-04-29 22:00:00,6.960227,64.847826,41.087963,22.029321,67.175608,7.142857,67.175608,79.846794
10031,2005-04-29 23:00:00,6.960227,72.304348,37.152778,22.029321,67.175608,7.142857,72.304348,78.651142


In [3]:
data_clean_path = 'clean.csv'
data_clean = pd.read_csv(data_clean_path)
data_clean

,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,DateTime,Season
0,1360.0,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578,2004-03-10 18:00:00,winter
1,1292.0,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255,2004-03-10 19:00:00,winter
2,1402.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502,2004-03-10 20:00:00,winter
3,1376.0,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867,2004-03-10 21:00:00,winter
4,1272.0,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888,2004-03-10 22:00:00,winter
...,...,...,...,...,...,...,...,...,...,...
9352,1314.0,1101.0,539.0,1374.0,1729.0,21.9,29.3,0.7568,2005-04-04 10:00:00,spring
9353,1163.0,1027.0,604.0,1264.0,1269.0,24.3,23.7,0.7119,2005-04-04 11:00:00,spring
9354,1142.0,1063.0,603.0,1241.0,1092.0,26.9,18.3,0.6406,2005-04-04 12:00:00,spring
9355,1003.0,961.0,702.0,1041.0,770.0,28.3,13.5,0.5139,2005-04-04 13:00:00,spring


In [4]:
data_clean['PT08.S1(CO)'] = data_clean['PT08.S1(CO)'] * 0.01
data_clean['PT08.S2(NMHC)'] = data_clean['PT08.S2(NMHC)'] * 0.01
data_clean['PT08.S3(NOx)'] = data_clean['PT08.S3(NOx)'] * 0.01
data_clean['PT08.S4(NO2)'] = data_clean['PT08.S4(NO2)'] * 0.01
data_clean['PT08.S5(O3)'] = data_clean['PT08.S5(O3)'] * 0.01
data_clean

,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,DateTime,Season
0,13.60,10.46,10.56,16.92,12.68,13.6,48.9,0.7578,2004-03-10 18:00:00,winter
1,12.92,9.55,11.74,15.59,9.72,13.3,47.7,0.7255,2004-03-10 19:00:00,winter
2,14.02,9.39,11.40,15.55,10.74,11.9,54.0,0.7502,2004-03-10 20:00:00,winter
3,13.76,9.48,10.92,15.84,12.03,11.0,60.0,0.7867,2004-03-10 21:00:00,winter
4,12.72,8.36,12.05,14.90,11.10,11.2,59.6,0.7888,2004-03-10 22:00:00,winter
...,...,...,...,...,...,...,...,...,...,...
9352,13.14,11.01,5.39,13.74,17.29,21.9,29.3,0.7568,2005-04-04 10:00:00,spring
9353,11.63,10.27,6.04,12.64,12.69,24.3,23.7,0.7119,2005-04-04 11:00:00,spring
9354,11.42,10.63,6.03,12.41,10.92,26.9,18.3,0.6406,2005-04-04 12:00:00,spring
9355,10.03,9.61,7.02,10.41,7.70,28.3,13.5,0.5139,2005-04-04 13:00:00,spring


In [5]:
aqi_data['AQI_NO2'] = aqi_data['AQI_NO2'] / 3
aqi_data['AQI_O3'] = aqi_data['AQI_O3'] / 2
aqi_data['AQI_PM25'] = aqi_data['AQI_PM25'] / 3.2
aqi_data['AQI_SO2'] = aqi_data['AQI_SO2'] * 1.5
aqi_data['AQI_PM10'] = aqi_data['AQI_PM10'] / 2.8
aqi_data

,Start,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25,AQI_SO2,Overall_AQI,Overall_AQI_Smoothed
0,2004-03-01 00:00:00,19.318182,20.195652,4.979817,20.335498,21.786481,12.857143,69.716738,NaN
1,2004-03-01 01:00:00,18.181818,17.355072,4.979817,20.335498,21.786481,0.000000,69.716738,NaN
2,2004-03-01 02:00:00,17.803030,14.465409,4.979817,20.335498,21.786481,10.714286,69.716738,NaN
3,2004-03-01 03:00:00,17.329546,11.320755,4.979817,20.335498,21.786481,0.000000,69.716738,NaN
4,2004-03-01 04:00:00,17.045455,12.264151,4.979817,20.335498,21.786481,0.000000,69.716738,NaN
...,...,...,...,...,...,...,...,...,...
10028,2005-04-29 20:00:00,7.528409,21.260870,21.412037,7.867615,20.992378,12.857143,67.175608,81.827631
10029,2005-04-29 21:00:00,7.244318,21.615942,21.498843,7.867615,20.992378,10.714286,67.175608,80.992557
10030,2005-04-29 22:00:00,6.960227,21.615942,20.543981,7.867615,20.992378,10.714286,67.175608,79.846794
10031,2005-04-29 23:00:00,6.960227,24.101449,18.576389,7.867615,20.992378,10.714286,72.304348,78.651142


In [8]:
merged_data = pd.concat([aqi_data, data_clean])
merged_data

,Start,AQI_CO,AQI_NO2,AQI_O3,AQI_PM10,AQI_PM25,AQI_SO2,Overall_AQI,Overall_AQI_Smoothed,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,DateTime,Season
0,2004-03-01 00:00:00,19.318182,20.195652,4.979817,20.335498,21.786481,12.857143,69.716738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-03-01 01:00:00,18.181818,17.355072,4.979817,20.335498,21.786481,0.000000,69.716738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-03-01 02:00:00,17.803030,14.465409,4.979817,20.335498,21.786481,10.714286,69.716738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-03-01 03:00:00,17.329546,11.320755,4.979817,20.335498,21.786481,0.000000,69.716738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-03-01 04:00:00,17.045455,12.264151,4.979817,20.335498,21.786481,0.000000,69.716738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.14,11.01,5.39,13.74,17.29,21.9,29.3,0.7568,2005-04-04 10:00:00,spring
9353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.63,10.27,6.04,12.64,12.69,24.3,23.7,0.7119,2005-04-04 11:00:00,spring
9354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.42,10.63,6.03,12.41,10.92,26.9,18.3,0.6406,2005-04-04 12:00:00,spring
9355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.03,9.61,7.02,10.41,7.70,28.3,13.5,0.5139,2005-04-04 13:00:00,spring
